In [35]:
import pandas as pd
import numpy as np

In [25]:
df = pd.read_csv(r'E:\Data Scientist\video_gaming_data_set\fifa21 raw data v2.csv', low_memory=False)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18979 entries, 0 to 18978
Data columns (total 77 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                18979 non-null  int64 
 1   Name              18979 non-null  object
 2   LongName          18979 non-null  object
 3   photoUrl          18979 non-null  object
 4   playerUrl         18979 non-null  object
 5   Nationality       18979 non-null  object
 6   Age               18979 non-null  int64 
 7   ↓OVA              18979 non-null  int64 
 8   POT               18979 non-null  int64 
 9   Club              18979 non-null  object
 10  Contract          18979 non-null  object
 11  Positions         18979 non-null  object
 12  Height            18979 non-null  object
 13  Weight            18979 non-null  object
 14  Preferred Foot    18979 non-null  object
 15  BOV               18979 non-null  int64 
 16  Best Position     18979 non-null  object
 17  Joined      

In [26]:
#Checking null datas
df.isna().sum().sort_values(ascending=False).head(10)

Loan Date End    17966
Hits              2595
LongName             0
Name                 0
ID                   0
Nationality          0
Age                  0
↓OVA                 0
POT                  0
Club                 0
dtype: int64

Players not on loan have missing dates, so I replaced them with ‘Not on Loan’ for clarity.
I filled missing Hits with the median value, keeping data consistent without biasing the averages.

In [32]:
df['Loan Date End'] = df['Loan Date End'].fillna('Not on loan')


df['Hits'] = pd.to_numeric(df['Hits'], errors='coerce')
df['Hits'] = df['Hits'].fillna(df['Hits'].median())

In [34]:
#looking at data
df.head(5)

,ID,Name,LongName,photoUrl,playerUrl,Nationality,Age,↓OVA,POT,Club,...,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits
0,158023,L. Messi,Lionel Messi,https://cdn.sofifa.com/players/158/023/21_60.png,http://sofifa.com/player/158023/lionel-messi/2...,Argentina,33,93,93,\n\n\n\nFC Barcelona,...,Medium,Low,5 ★,85,92,91,95,38,65,771.0
1,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,https://cdn.sofifa.com/players/020/801/21_60.png,http://sofifa.com/player/20801/c-ronaldo-dos-s...,Portugal,35,92,92,\n\n\n\nJuventus,...,High,Low,5 ★,89,93,81,89,35,77,562.0
2,200389,J. Oblak,Jan Oblak,https://cdn.sofifa.com/players/200/389/21_60.png,http://sofifa.com/player/200389/jan-oblak/210006/,Slovenia,27,91,93,\n\n\n\nAtlético Madrid,...,Medium,Medium,3 ★,87,92,78,90,52,90,150.0
3,192985,K. De Bruyne,Kevin De Bruyne,https://cdn.sofifa.com/players/192/985/21_60.png,http://sofifa.com/player/192985/kevin-de-bruyn...,Belgium,29,91,91,\n\n\n\nManchester City,...,High,High,4 ★,76,86,93,88,64,78,207.0
4,190871,Neymar Jr,Neymar da Silva Santos Jr.,https://cdn.sofifa.com/players/190/871/21_60.png,http://sofifa.com/player/190871/neymar-da-silv...,Brazil,28,91,91,\n\n\n\nParis Saint-Germain,...,High,Medium,5 ★,91,85,86,94,36,59,595.0


Columns like Value, Wage, Release Clause may look like:
€120.5M, €200K, etc.
We will convert them into numeric.

In [36]:
def converter(value):
    value = str(value).replace('€', '').replace('M', 'e6').replace('K', 'e3')
    try:
        return eval(value)
    except:
        return np.nan
    
df['Value'] = df['Value'].apply(converter)
df['Wage'] = df['Wage'].apply(converter)
df['Release Clause'] = df['Release Clause'].apply(converter)

Clean Height and Weight
They might be in formats like "6'2" and "180lbs" convert to centimeters and kilograms:

In [38]:
def height_to_cm(x):
    try:
        feet, inch = x.split("'")
        return int(feet)*30.48 + int(inch)*2.54
    except:
        return np.nan 
    
def weight_to_kg(x):
    try:
        return int(x.replace('lbs', '')) * 0.453592
    except:
        return np.nan
    
df['Height(cm)'] = df['Height'].apply(height_to_cm)
df['Weight(kg)'] = df['Weight'].apply(weight_to_kg)

In [ ]:
#Convert Joined, Loan Date End, etc., into datetime format.

df['Joined'] = pd.to_datetime(df['Joined'], errors='coerce')
df['Loan Date End'] = pd.to_datetime(df['Loan Date End'], errors='coerce')


In [39]:
# Make sure columns like Nationality, Club, Preferred Foot have consistent formatting:
df['Nationality'] = df['Nationality'].str.strip().str.title()
df['Preferred Foot'] = df['Preferred Foot'].str.strip().str.capitalize()


In [40]:
#Removing duplicates
df = df.drop_duplicates()


In [41]:
df.to_csv('fifa21_cleaned.csv', index=False)
